## Load the meta data into a dataframe

In [ ]:
import pandas as pd

df = pd.read_excel('../data/detector_tests/labelled_photos_file_paths.xlsx')

df


## Plot the locations of the camera traps on a map 

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np

# Convert 'na' strings to NaN and drop rows with NaN
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df_clean = df.dropna(subset=['Longitude', 'Latitude'])

# Select unique locations based on Longitude and Latitude
unique_locations = df_clean[['Longitude', 'Latitude']].drop_duplicates()


In [ ]:

# Create a GeoDataFrame, specifying the coordinate reference system (CRS) as WGS84 (epsg:4326)
gdf = gpd.GeoDataFrame(unique_locations, geometry=gpd.points_from_xy(unique_locations.Longitude, unique_locations.Latitude))
gdf.crs = "EPSG:4326"

# Convert the GeoDataFrame to Web Mercator projection (epsg:3857) for use with contextily
gdf = gdf.to_crs(epsg=3857)

# Plotting
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, color='red', markersize=5)
ctx.add_basemap(ax, source=ctx.providers.OpenTopoMap)
ax.set_axis_off()
plt.show()


## Plot a bar char of the count for each species

In [ ]:
# Count the occurrences of each species
species_counts = df['Species'].value_counts()

# Plot the counts as a bar chart
plt.figure(figsize=(10, 8))  # Adjust the size as needed
ax = species_counts.plot(kind='bar')
plt.title('Frequency of Each Species')
plt.xlabel('Species')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha="right")
plt.tight_layout() 

# Add count labels to the top of the bars
for i, v in enumerate(species_counts):
    ax.text(i, v, str(v), ha='center', va='bottom')

plt.show()


## Plot the frequency of detections of wild boar on a map

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

# Filter for Wild boar
wild_boar_df = df[df['Species'] == 'Wild boar']

# Count detections at each unique location
location_counts = wild_boar_df.groupby(['Longitude', 'Latitude']).size().reset_index(name='Counts')

# Create a GeoDataFrame from the filtered data
gdf = gpd.GeoDataFrame(location_counts, geometry=gpd.points_from_xy(location_counts.Longitude, location_counts.Latitude))

# Set the coordinate reference system (CRS) to WGS84
gdf.crs = 'EPSG:4326'

# Convert the GeoDataFrame to Web Mercator projection
gdf = gdf.to_crs(epsg=3857)

# Plotting
fig, ax = plt.subplots(figsize=(10, 10))

# Adjust marker size based on the count of detections.
gdf.plot(ax=ax, markersize=gdf['Counts']*10, color='red', alpha=0.5)

# Add a basemap
ctx.add_basemap(ax, source=ctx.providers.OpenTopoMap)

# Remove axis
ax.set_axis_off()

plt.show()

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import box, Point
import matplotlib.patches as patches

# Filter for Wild boar
wild_boar_df = df[df['Species'] == 'Wild boar']

# Count detections at each unique location
location_counts = wild_boar_df.groupby(['Longitude', 'Latitude']).size().reset_index(name='Counts')

# Create a GeoDataFrame from the filtered data
gdf = gpd.GeoDataFrame(location_counts, geometry=gpd.points_from_xy(location_counts.Longitude, location_counts.Latitude))

# Set the coordinate reference system (CRS) to WGS84
gdf.crs = 'EPSG:4326'

# Define the camera trap sites data
trap_sites_data = {
    'X': [25.01059634, 25.00330022, 25.09574386],
    'Y': [45.326467, 45.32415226, 45.32770572],
    'Name': ['Piscul Lat', 'Albești', 'Feeding Site']
}

# Create a DataFrame for the trap sites
trap_sites_df = pd.DataFrame(trap_sites_data)

# Create a GeoDataFrame from the trap sites data
trap_sites_gdf = gpd.GeoDataFrame(trap_sites_df, geometry=gpd.points_from_xy(trap_sites_df.X, trap_sites_df.Y))

# Set the CRS for the trap sites GeoDataFrame to WGS84
trap_sites_gdf.crs = 'EPSG:4326'

# Convert the trap sites GeoDataFrame to Web Mercator projection
trap_sites_gdf = trap_sites_gdf.to_crs(epsg=3857)

# Define the towns data
towns_data = {
    'X': [25.0940, 25.1761, 25.0696, 25.00666789592026],
    'Y': [45.2935, 45.3893, 45.3342, 45.30880113705865],
    'Name': ['Valea Mare-Pravăț', 'Rucăr', 'Lerești', 'Albești']
}

# Create a DataFrame for the towns
towns_df = pd.DataFrame(towns_data)

# Create a GeoDataFrame from the towns data
towns_gdf = gpd.GeoDataFrame(towns_df, geometry=gpd.points_from_xy(towns_df.X, towns_df.Y))

# Set the CRS for the towns GeoDataFrame to WGS84
towns_gdf.crs = 'EPSG:4326'

# Convert the towns GeoDataFrame to Web Mercator projection
towns_gdf = towns_gdf.to_crs(epsg=3857)

# Calculate the northernmost, southernmost, westernmost, and easternmost bounds from the towns
north_bound = towns_df['Y'].max()
south_bound = towns_df['Y'].min()
west_bound = towns_df['X'].min()
east_bound = towns_df['X'].max()

# Filter the unique locations to be within the bounds
filtered_gdf = gdf[(gdf['Latitude'] <= north_bound) & (gdf['Latitude'] >= south_bound) &
                   (gdf['Longitude'] >= west_bound) & (gdf['Longitude'] <= east_bound)].to_crs(epsg=3857)

# Assuming conflict_df is the DataFrame with conflict data
# Convert conflict_df into a GeoDataFrame
conflict_gdf = gpd.GeoDataFrame(
    conflict_df,
    geometry=gpd.points_from_xy(conflict_df.x, conflict_df.y)
)

# Set the CRS for the conflict GeoDataFrame to WGS84
conflict_gdf.crs = 'EPSG:4326'

# Filter the conflict data to be within the bounds
filtered_conflict_gdf = conflict_gdf[(conflict_gdf['y'] <= north_bound) & (conflict_gdf['y'] >= south_bound) &
                                     (conflict_gdf['x'] >= west_bound) & (conflict_gdf['x'] <= east_bound)].to_crs(epsg=3857)

# Plotting
fig, ax = plt.subplots(figsize=(12, 12))

# Plot the existing camera traps as small dots
existing_camera_traps = filtered_gdf.plot(ax=ax, markersize=30, color='black', marker='o', alpha=0.7, label='Existing camera traps')

# Plot the trap sites with different markers
alert_system_deployments = trap_sites_gdf.plot(ax=ax, markersize=100, color='blue', marker='o', alpha=1.0, label='Alert system deployments')

# Plot the towns with different markers
study_villages = towns_gdf.plot(ax=ax, markersize=100, color='black', marker='*', alpha=1.0, label='Study villages')

# Plot the filtered conflict data
conflict_occurrence = filtered_conflict_gdf.plot(ax=ax, markersize=50, color='red', marker='x', alpha=0.7, label='Conflict occurrence')

# Add labels for the trap sites with special handling for "Piscul Lat"
for x, y, label in zip(trap_sites_gdf.geometry.x, trap_sites_gdf.geometry.y, trap_sites_gdf['Name']):
    if label == 'Piscul Lat':
        ax.text(x, y + 200, label, fontsize=14, ha='center', color='black', fontweight='bold')  # Position above
    else:
        ax.text(x, y - 500, label, fontsize=14, ha='center', color='black', fontweight='bold')  # Position below

# Add labels for the towns
for x, y, label in zip(towns_gdf.geometry.x, towns_gdf.geometry.y, towns_gdf['Name']):
    ax.text(x, y - 500, label, fontsize=14, ha='center', color='black', fontweight='bold')  # Position below

# Add a high-resolution basemap
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=14)

# Remove axis
ax.set_axis_off()

# Manually create the legend with the desired order
handles, labels = ax.get_legend_handles_labels()
order = [2, 3, 0, 1]  # Order: Study villages, Conflict occurrence, Existing camera traps, Alert system deployments
legend = ax.legend([handles[idx] for idx in order], [labels[idx] for idx in order], fontsize=14, title_fontsize=14, loc='upper left')

for text in legend.get_texts():
    text.set_fontweight('bold')

# Add an inset map
inset_ax = fig.add_axes([0, 0.52, 0.3, 0.3])

romania = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')).query("name == 'Romania'")
romania.to_crs(epsg=3857).plot(ax=inset_ax, color='green', edgecolor='black', zorder=1)

# Add Bucharest to the inset map
bucharest = gpd.GeoDataFrame(geometry=[Point(26.1025, 44.4268)], crs='EPSG:4326').to_crs(epsg=3857)
bucharest.plot(ax=inset_ax, marker='*', color='black', markersize=100, zorder=2)

# Add the larger map extent to the inset map with a red square marker
main_map_extent = box(west_bound, south_bound, east_bound, north_bound)
main_map_gdf = gpd.GeoDataFrame(geometry=[main_map_extent], crs='EPSG:4326').to_crs(epsg=3857)
main_map_gdf.boundary.plot(ax=inset_ax, color='black', linewidth=1, zorder=2)

# Add Romania label to the inset map
inset_ax.text(0.5, 0.65, 'Romania', fontsize=14, ha='center', fontweight='bold', transform=inset_ax.transAxes, zorder=3)

inset_ax.text(0.67, 0.35, 'Study area', fontsize=10, ha='center', fontweight='bold', transform=inset_ax.transAxes, zorder=3)

inset_ax.text(0.755, 0.173, 'Bucharest', fontsize=10, ha='center', fontweight='bold', transform=inset_ax.transAxes, zorder=3)

# Remove axis for inset map
inset_ax.set_axis_off()

# Remove white border
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

plt.show()


In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap

# Load the data from the Excel file into a DataFrame
conflict_df = pd.read_excel('../data/conflict_coordinates/ConflictReporterCarpathia_layer.xlsx')

# Create a base map
m = folium.Map(location=[conflict_df['y'].mean(), conflict_df['x'].mean()], zoom_start=8)

# Prepare the data for the heat map
heat_data = [[row['y'], row['x']] for index, row in conflict_df.iterrows()]

# Define a muted color gradient using reds and yellows
gradient = {0.2: 'lightyellow', 0.4: 'khaki', 0.6: 'gold', 0.8: 'salmon', 1: 'lightcoral'}

# Add the heat map with the muted red and yellow gradient
HeatMap(heat_data, gradient=gradient).add_to(m)

# Display the map
m


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Load the world map from geopandas datasets
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Filter for Romania
romania = world[world.name == "Romania"]

# Coordinates for Bucharest
bucharest_coords = (26.1025, 44.4268)

# Create a GeoDataFrame for Bucharest
bucharest = gpd.GeoDataFrame(
    {'City': ['Bucharest']},
    geometry=[Point(bucharest_coords)],
    crs="EPSG:4326"
)

# Plot Romania
fig, ax = plt.subplots(figsize=(10, 10))
romania.boundary.plot(ax=ax, color='black')  # Add black border around Romania
romania.plot(ax=ax, color='lightgreen', edgecolor='black')  # Fill Romania with light green color

# Plot Bucharest with a star marker
bucharest.plot(ax=ax, color='red', marker='*', markersize=200)

# Set the background to transparent
fig.patch.set_alpha(0)
ax.patch.set_alpha(0)

# Remove axis
ax.axis('off')

# Show plot
plt.show()
